# LLM训练全链路最佳实践

随着人工智能技术的飞速发展，大型语言模型（LLMs）已经成为自然语言处理领域的核心驱动力。本文档旨在概述使用modelscope生态进行LLM训练的全链路最佳实践，涵盖数据下载、数据预处理、模型训练、模型评估完整流程。

主要内容

教程以知乎评论数据集为例，使用LoRA微调模型，让AI生成的文本没有那么强的“AI味”

本教程涉及以下框架的安装和使用：
1. modelscope：提供模型、数据集下载能力 
2. data-juicer：提供数据集处理能力
1. ms-swift：提供模型训练、推理能力
1. evalscope：提供模型评测能力

## 环境准备

安装modelscope、data-juicer、swift、evalscope

In [2]:
# %pip install modelscope[framework]  # 模型库，notebook已预装
%pip install ms-swift[llm] -U          # 训练库
%pip install evalscope -U             # 评测库
%pip install py-data-juicer[sci]      # 数据处理库
%pip install datasets==3.0.1 pydantic==2.0 tf-keras
%pip uninstall tensorflow -y    # 不需要，跟环境冲突

Found existing installation: tensorflow 2.18.0
Uninstalling tensorflow-2.18.0:
  Successfully uninstalled tensorflow-2.18.0
Note: you may need to restart the kernel to use updated packages.


# ！！重启notebook环境！！
------

## 数据集准备

使用modelscope下载数据集，初步处理数据集，提取需要的字段，并处理成data-juicer需要的格式

In [1]:
from modelscope import MsDataset
from pprint import pprint

ds =  MsDataset.load('OmniData/Zhihu-KOL', cache_dir="data", split='train')
print(ds)
pprint(ds[0])

/usr/local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Dataset({
    features: ['INSTRUCTION', 'RESPONSE', 'SOURCE', 'METADATA'],
    num_rows: 1006218
})
{'INSTRUCTION': '怎么说服男朋友买烤箱？',
 'METADATA': '{"question_id": 357137111.0, "answer_id": 914332816.0, "url": '
             '"https://www.zhihu.com/question/357137111/answer/914332816", '
             '"upvotes": "赞同 15", "answer_creation_time": '
             '"2019-11-28T12:01:22.000Z"}',
 'RESPONSE': 'emmmmm，首先想说的是，我买厨房用品一般是不用「说服」的，只是在厨房堆的满满当当的情况下会象征性的问一下我老公，他就会回答我说：你看看你还有地方放吗。然后我会思考一下，如果是特别想买的，就不会问他了。自己决定就好。 '
             '比如，前几天我又买了两个盘子~~~~他还不知道。 可以给题主看看我有多少的锅具：自家炒菜用什么锅好？各有什么优缺点？ '
             '说回烤箱的问题，买的时候处于热恋期，我告诉他我有一个买烤箱的计划。虽然他基本不吃点心，也不喜欢烘焙，但那个时期的他欣然同意并热情洋溢的给我选烤箱。可能是他有憧憬我会给他做什么好吃的吧。又因为我是一个不怎么吃甜食的湖南人，烤箱在我家烘焙的使用率很低。 '
             '但是！！你还是可以告诉他烤箱的作用是可以烤制各种肉类！！！我不相信有不喜欢吃肉的男生！！烤箱真的是可以烤一切的肉类，熟悉之后会觉得非常简单。 '
             '我很久以前用烤箱做的最多的就是烤羊排和烤鸡翅，我老公不怎么吃羊肉和鸡翅。这个烤箱因为厨房放不下，被放在了餐厅，也就闲置了下来…… '
             '要说的事是，烤箱真的能给你做出很多不一样的美食，尤其是来了客人，在你两个灶台忙不过来的时候，烤箱特别适合准备一个荤素搭配的豪华大菜。在烹饪其他需要爆炒的菜肴的空档去处理一下

In [2]:
# 处理 metadata
import json
# load json
metadata = list(map(lambda x: json.loads(x), ds['METADATA']))

# 处理 upvotes 
vote_list = []
for item in metadata:
    try:
        upvotes = item['upvotes'][3:]
        if not upvotes:
            votes = 0
        elif '万' in upvotes:
            votes = int(float(upvotes[:-2]) * 10000)
        else:
            votes = int(upvotes)
    except Exception as e:
        print(upvotes)
        votes = 0
    vote_list.append(votes)
print("Done")

Done


In [3]:
# 写入 jsonl 文件
import pandas as pd

df = pd.DataFrame.from_dict({
    'query': ds['INSTRUCTION'],
    'response': ds['RESPONSE'],
    'upvotes': vote_list
})

print(len(df))

df.to_json("data/zhihu.jsonl", orient="records", lines=True, force_ascii=False)
df.head()

1006218


,query,response,upvotes
0,怎么说服男朋友买烤箱？,emmmmm，首先想说的是，我买厨房用品一般是不用「说服」的，只是在厨房堆的满满当当的情况下...,15
1,航天从业者是如何看待电视剧《你是我的荣耀》的？,难得有个关于航天的剧，职场情节悬不悬浮，航天设定和细节走不走心？带着放大镜看了前18集，...,4432
2,如何看待PayPal正式进入中国？,PayPal不仅是美国支付巨头，也是国际支付巨头，目前已开拓全球200多个市场，美国以外的市...,127
3,中金公司交易员月薪八万五是如何做到的？,1、首先，考虑到这位交易员的工作经验，月薪八万五的表述是不正确的：其实是一年的全部薪酬除以1...,450
4,摇滚乐（金属）给你们带来了什么？,"ㄟ( ▔, ▔ )ㄏ哪里带来了什么东西啊，除了找到热爱的东西，也失去了很多。听重型现场像疯子...",5


## 使用data-juicer进行数据清洗


> Data-Juicer 是一个一站式多模态数据处理系统，旨在为大语言模型 (LLM) 提供更高质量、更丰富、更易“消化”的数据。设计简单易用，提供全面的文档、简易入门指南和演示配置，并且可以轻松地添加/删除现有配置中的算子。

详细介绍：https://github.com/modelscope/data-juicer/blob/main/README_ZH.md


### 1. 编写yaml配置文件

支持的算子：https://github.com/modelscope/data-juicer/blob/main/docs/Operators_ZH.md

| 类型                                | 数量 | 描述            |
|------------------------------------|:--:|---------------|
| [ Formatter ]( #formatter )        |  7 | 发现、加载、规范化原始数据 |
| [ Mapper ]( #mapper )              | 43 | 对数据样本进行编辑和转换  |
| [ Filter ]( #filter )              | 41 | 过滤低质量样本       |
| [ Deduplicator ]( #deduplicator )  |  5 | 识别、删除重复样本     |
| [ Selector ]( #selector )          |  4 | 基于排序选取高质量样本   |

在[全部算子的配置文件](https://github.com/modelscope/data-juicer/blob/main/configs/config_all.yaml)的基础上进行修改，编写如下配置文件：

**请手动创建该`zhihu-bot.yaml`，放在当前目录下**

```yaml

# global parameters
project_name: 'zhihu-process'
dataset_path: 'data/zhihu.jsonl'                            # path to your dataset directory or file
np: 16                                                      # number of subprocess to process your dataset

text_keys: 'response'                                       # the key of text in your dataset file

export_path: 'data/zhihu_refine.jsonl'                      # path to save processed dataset

# process schedule
# a list of several process operators with their arguments
process:
  - specified_numeric_field_filter:                         # filter text with the specified numeric field info out of specific range
      field_key: 'upvotes'                                      # the target key corresponding to multi-level field information need to be separated by '.'
      min_value: 500                                            # the min filter value in SpecifiedNumericField op
  - text_length_filter:                                     # filter text with the length out of specific range
      min_len: 100
      max_len: 2000

  - clean_email_mapper:                                     # remove emails from text.
  - clean_html_mapper:                                      # remove html formats form text.
  - clean_ip_mapper:                                        # remove ip addresses from text.
  - clean_links_mapper:                                     # remove web links from text.
  - clean_copyright_mapper:                                 # remove copyright comments.                              # fix unicode errors in text.

  - language_id_score_filter:                               # filter text in specific language with language scores larger than a specific max value
      lang: zh
      min_score: 0.9
  - alphanumeric_filter:                                    # filter text with alphabet/numeric ratio out of specific range.  
      tokenization: false
      min_ratio: 0.72
  - flagged_words_filter:                                   # filter text with the flagged-word ratio larger than a specific max value
      lang: zh
      tokenization: false
      max_ratio: 0.0005  
  - perplexity_filter:                                      # filter text with perplexity score out of specific range
      lang: zh
      max_ppl: 4000
  - special_characters_filter:                              # filter text with special-char ratio out of specific range
      max_ratio: 0.4  
  - document_simhash_deduplicator:                          # deduplicate texts with simhash
      tokenization: character
      window_size: 5  
      lowercase: false
      ignore_pattern: '\p{P}'
      num_blocks: 10
      hamming_distance: 6                                   # larger hamming distance threshold for short texts
  - topk_specified_field_selector:                          # selector to select top samples based on the sorted specified field
      field_key: 'upvotes'                                    # the target keys corresponding to multi-level field information need to be separated by '.'
      topk: 50000                                             # number of selected top sample
      reverse: True                                           # determine the sorting rule, if reverse=True, then sort in descending order
```

### 2. 根据配置文件进行数据分析 

In [ ]:
!dj-analyze --config zhihu-bot.yaml 

#### 数据集分析结果

- 箱型图
- 直方图
- 统计信息

在`data/analysis`路径下

|        |   alnum_ratio |   flagged_words_ratio | lang      |    lang_score |     perplexity |   special_char_ratio |         text_len |
|:-------|--------------:|----------------------:|:----------|--------------:|---------------:|---------------------:|-----------------:|
| count  |   1.00622e+06 |           1.00622e+06 | 1006218.0 |   1.00622e+06 |    1.00622e+06 |          1.00622e+06 |      1.00622e+06 |
| mean   |   0.871938    |           1.28188e-05 | nan       |   0.963631    | 2390           |          0.159879    |    717.802       |
| std    |   0.0793817   |           0.00120551  | nan       |   0.0976119   | 4733.66        |          0.0878637   |   1666.89        |
| min    |   0           |           0           | nan       |   0.0593122   |    0           |          0           |      1           |
| 25%    |   0.854922    |           0           | nan       |   0.976512    | 1500.4         |          0.118577    |     61           |
| 50%    |   0.883008    |           0           | nan       |   0.989479    | 2017.7         |          0.147059    |    236           |
| 75%    |   0.905219    |           0           | nan       |   0.994992    | 2695.5         |          0.183099    |    764           |
| max    |   1           |           0.6         | nan       |   1.00007     |    1.70447e+06 |          1           | 139406           |
| unique | nan           |         nan           | 99.0      | nan           |  nan           |        nan           |    nan           |
| top    | nan           |         nan           | zh        | nan           |  nan           |        nan           |    nan           |
| freq   | nan           |         nan           | 990697.0  | nan           |  nan           |        nan           |    nan           |


### 3. 调整配置文件进行数据处理

这一步的数据处理包括：筛选、过滤、去重 

根据分析得到的数据集特征，调整配置文件，再进行数据处理:

- 数据处理3σ法则：若某个数据点超出均值±3σ的范围，通常被视为异常值
- 先进行筛选，再过滤，能减少数据处理的时间

In [ ]:
!dj-process --config zhihu-bot.yaml 

### 4. 划分训练集和测试集

In [6]:
import pandas as pd

data = pd.read_json("data/zhihu_refine.jsonl", lines=True)

def split_data(data, save=False, suffix=''):
    # split data into train and test, 9: 1
    train_data = data.sample(frac=0.9, random_state=42)
    test_data = data.drop(train_data.index)

    if suffix:
        suffix = '_' + suffix
    if save:
        train_data.to_json(f"data/zhihu_train{suffix}.jsonl", orient='records', lines=True, force_ascii=False)
        test_data.to_json(f"data/zhihu_test{suffix}.jsonl", orient='records', lines=True,  force_ascii=False)
    return train_data, test_data

train_data, test_data = split_data(data, save=True)

print(len(train_data))
print(len(test_data))

45000
5000


## 使用ms-swift训练模型


> SWIFT支持300+ LLM和50+ MLLM（多模态大模型）的训练(预训练、微调、对齐)、推理、评测和部署。开发者可以直接将我们的框架应用到自己的Research和生产环境中，实现模型训练评测到应用的完整链路。我们除支持了PEFT提供的轻量训练方案外，也提供了一个完整的Adapters库以支持最新的训练技术，如NEFTune、LoRA+、LLaMA-PRO等，这个适配器库可以脱离训练脚本直接使用在自己的自定流程中。

详细介绍：https://github.com/modelscope/ms-swift/blob/main/README_CN.md

(可选)安装 flash-attention 加快推理速度：

In [ ]:
%pip install flash-attn --no-build-isolation

### 编写训练脚本
命令行参数：https://swift.readthedocs.io/zh-cn/latest/Instruction/%E5%91%BD%E4%BB%A4%E8%A1%8C%E5%8F%82%E6%95%B0.html

需要注意的参数有：

1. dataset 可以混合一些通用数据集，防止模型灾难性遗忘和通用能力丢失
2. system可以设置一个符合任务特性的system prompt，提升模型能力
3. lora_target_modules可以根据训练任务的难易程度，调整可以训练的参数数量

In [5]:
!swift sft \
    --train_type lora \
    --model Qwen/Qwen2.5-0.5B-Instruct \
    --dataset data/zhihu_train.jsonl#3000 AI-ModelScope/Magpie-Qwen2-Pro-200K-Chinese#3000 \
    --system "你是一个真实的人类。回答用户的问题，并在聊天回复中展现你的情绪，包括开心，幸福，愤怒，伤心，阴阳怪气等" \
    --output_dir output \
    --target_modules all-linear \
    --lora_rank 4 \
    --torch_dtype bfloat16 \
    --seed 42 \
    --learning_rate 1e-4 \
    --warmup_ratio 0.05 \
    --max_length 1024 \
    --per_device_train_batch_size 4 \
    --per_device_eval_batch_size 4 \
    --num_train_epochs 1 \
    --gradient_accumulation_steps 4 \
    --save_total_limit 10 \
    --eval_steps 100 \
    --save_steps 100 

run sh: `/usr/local/bin/python /usr/local/lib/python3.10/site-packages/swift/cli/sft.py --train_type lora --model Qwen/Qwen2.5-0.5B-Instruct --dataset data/zhihu_train.jsonl#3000 AI-ModelScope/Magpie-Qwen2-Pro-200K-Chinese#3000 --system 你是一个真实的人类。回答用户的问题，并在聊天回复中展现你的情绪，包括开心，幸福，愤怒，伤心，阴阳怪气等 --output_dir output --target_modules all-linear --lora_rank 4 --torch_dtype bfloat16 --seed 42 --learning_rate 1e-4 --warmup_ratio 0.05 --max_length 1024 --per_device_train_batch_size 4 --per_device_eval_batch_size 4 --num_train_epochs 1 --gradient_accumulation_steps 4 --save_total_limit 10 --eval_steps 100 --save_steps 100`
[INFO:swift] Successfully registered `/usr/local/lib/python3.10/site-packages/swift/llm/dataset/data/dataset_info.json`
[INFO:swift] Successfully registered `[]`
[INFO:swift] rank: -1, local_rank: -1, world_size: 1, local_world_size: 1
[INFO:swift.hub.hub] Downloading the model from ModelScope Hub, model_id: Qwen/Qwen2.5-0.5B-Instruct
[WARNING:modelscope] Using branch: master as ve

### 导出合并lora后的模型

将lora合并回原本模型，得到训练之后的完整模型

In [7]:
!swift export \
    --adapters /mnt/workspace/output/v2-20241223-200525/checkpoint-340 \
    --merge_lora true

run sh: `/usr/local/bin/python /usr/local/lib/python3.10/site-packages/swift/cli/export.py --adapters /mnt/workspace/output/v2-20241223-200525/checkpoint-340 --merge_lora true`
[INFO:swift] Successfully registered `/usr/local/lib/python3.10/site-packages/swift/llm/dataset/data/dataset_info.json`
[INFO:swift] Loading the model using model_dir: /mnt/workspace/output/v2-20241223-200525/checkpoint-340
[INFO:swift] Successfully loaded /mnt/workspace/output/v2-20241223-200525/checkpoint-340/args.json.
[INFO:swift] Successfully registered `[]`
[INFO:swift] rank: -1, local_rank: -1, world_size: 1, local_world_size: 1
[INFO:swift.hub.hub] Downloading the model from ModelScope Hub, model_id: Qwen/Qwen2.5-0.5B-Instruct
[WARNING:modelscope] Using branch: master as version is unstable, use with caution
[INFO:modelscope] Target directory already exists, skipping creation.
[INFO:swift] Loading the model using model_dir: /mnt/workspace/.cache/modelscope/hub/Qwen/Qwen2___5-0___5B-Instruct
[INFO:swift] 

## 使用evalscope评估模型

> EvalScope是魔搭社区官方推出的模型评测与性能基准测试框架，专为多样化的模型评估需求而设计。它支持广泛的模型类型，包括但不限于大语言模型、多模态模型、Embedding 模型、Reranker 模型和 CLIP 模型。

详细介绍：https://github.com/modelscope/evalscope/blob/main/README_zh.md


  


### 1. 评估模型通用能力

EvalScope 集成了多个数据集，可以用来评测模型的通用能力，包括数学能力、推理能力等，下面我们使用ARC数据集测试模型的推理能力

In [8]:
!evalscope eval \
  --model output/v2-20241223-200525/checkpoint-340-merged \
  --datasets arc

2024-12-23 20:22:04,539 - datasets - INFO - PyTorch version 2.4.0 available.
2024-12-23 20:22:04,540 - datasets - INFO - Polars version 1.16.0 available.
2024-12-23 20:22:06,854 - evalscope - INFO - Args: Task config is provided with CommandLine type.
2024-12-23 20:22:06,890 - evalscope - INFO - Dump task config to ./outputs/20241223_202206/configs/task_config_16e22f.yaml
2024-12-23 20:22:06,903 - evalscope - INFO - {
    "model": "output/v2-20241223-200525/checkpoint-340-merged",
    "model_id": "checkpoint-340-merged",
    "model_args": {
        "revision": "master",
        "precision": "torch.float16",
        "device": "auto"
    },
    "template_type": null,
    "chat_template": null,
    "datasets": [
        "arc"
    ],
    "dataset_args": {},
    "dataset_dir": "/mnt/workspace/.cache/modelscope/datasets",
    "dataset_hub": "modelscope",
    "generation_config": {
        "max_length": 2048,
        "max_new_tokens": 512,
        "do_sample": false,
        "top_k": 50,
    

下面看一下原始模型的推理能力

In [9]:
!evalscope eval \
  --model Qwen/Qwen2.5-0.5B-Instruct \
  --datasets arc

2024-12-23 20:24:33,485 - datasets - INFO - PyTorch version 2.4.0 available.
2024-12-23 20:24:33,485 - datasets - INFO - Polars version 1.16.0 available.
2024-12-23 20:24:36,022 - evalscope - INFO - Args: Task config is provided with CommandLine type.
2024-12-23 20:24:36,049 - evalscope - INFO - Dump task config to ./outputs/20241223_202436/configs/task_config_a88c00.yaml
2024-12-23 20:24:36,062 - evalscope - INFO - {
    "model": "Qwen/Qwen2.5-0.5B-Instruct",
    "model_id": "Qwen2.5-0.5B-Instruct",
    "model_args": {
        "revision": "master",
        "precision": "torch.float16",
        "device": "auto"
    },
    "template_type": null,
    "chat_template": null,
    "datasets": [
        "arc"
    ],
    "dataset_args": {},
    "dataset_dir": "/mnt/workspace/.cache/modelscope/datasets",
    "dataset_hub": "modelscope",
    "generation_config": {
        "max_length": 2048,
        "max_new_tokens": 512,
        "do_sample": false,
        "top_k": 50,
        "top_p": 1.0,
   

可以看出，通过自定义微调后模型的推理能力有些许下降，也可以接受。

### 2. 自定义数据集评估

使用general qa模版自定义评估数据集

**评估指标：**
- bleu：比较生成文本和参考文本中的n-gram（n个连续单词的序列）。常见的n有1（unigram）、2（bigram）、3（trigram）等。
- rouge： 侧重于召回率（recall）

**数据格式：**

需要query和response两个字段，例如：
```json
{
  "query": "什么是机器学习？",
  "response": "机器学习（Machine Learning）是计算机科学的一个分支，它研究计算机如何根据已有的例子来学习，从而实现对未知数据的预测和分类。"
}
```   

In [16]:
!evalscope eval \
  --model output/v2-20241223-200525/checkpoint-340-merged \
  --datasets general_qa \
  --dataset-args '{"general_qa": {"local_path": "data", "subset_list": ["zhihu_test.jsonl"]}}' \
  --limit 10

2024-12-23 20:36:09,212 - datasets - INFO - PyTorch version 2.4.0 available.
2024-12-23 20:36:09,213 - datasets - INFO - Polars version 1.16.0 available.
2024-12-23 20:36:11,516 - evalscope - INFO - Args: Task config is provided with CommandLine type.
2024-12-23 20:36:11,544 - evalscope - INFO - Dump task config to ./outputs/20241223_203611/configs/task_config_ba7e37.yaml
2024-12-23 20:36:11,558 - evalscope - INFO - {
    "model": "output/v2-20241223-200525/checkpoint-340-merged",
    "model_id": "checkpoint-340-merged",
    "model_args": {
        "revision": "master",
        "precision": "torch.float16",
        "device": "auto"
    },
    "template_type": null,
    "chat_template": null,
    "datasets": [
        "general_qa"
    ],
    "dataset_args": {
        "general_qa": {
            "local_path": "data",
            "subset_list": [
                "zhihu_test.jsonl"
            ]
        }
    },
    "dataset_dir": "/mnt/workspace/.cache/modelscope/datasets",
    "dataset_h

## 模型上传

您可以使用modelscope [modelhub](https://modelscope.cn/docs/models/upload)来将已经训练好的模型上传到ModelScope平台。您可以提前在ModelScope社区网页创建对应模型，然后将本地模型目录通过push_model接口进行上传，也可以直接通过push_model自动完成模型创建和上传

In [11]:
from modelscope.hub.api import HubApi

YOUR_ACCESS_TOKEN = '请从ModelScope个人中心->访问令牌获取'

api = HubApi()
api.login(YOUR_ACCESS_TOKEN)
api.push_model(
    model_id="YOUR_NAME/zhihu_bot_lora", 
    model_dir="output/qwen2-7b-instruct/v1-20240819-150005/checkpoint-371" # 本地模型目录，要求目录中必须包含configuration.json
)

[ERROR:modelscope] The request model: AlexEz/zhihu_bot_lora does not exist!
[INFO:modelscope] Create new model AlexEz/zhihu_bot_lora
[INFO:modelscope] [master edb24d9] 'upload model'
 12 files changed, 1340 insertions(+), 47 deletions(-)
 rewrite README.md (98%)
 create mode 100644 adapter_config.json
 create mode 100644 adapter_model.safetensors
 create mode 100644 additional_config.json
 create mode 100644 configuration.json
 create mode 100644 generation_config.json
 create mode 100644 optimizer.pt
 create mode 100644 rng_state.pth
 create mode 100644 scheduler.pt
 create mode 100644 sft_args.json
 create mode 100644 trainer_state.json
 create mode 100644 training_args.bin

